# Extracción Source Data

In [1]:
import pandas as pd

url = "https://raw.githubusercontent.com/JosefRivers-Dev/Data-Science_Telecom_2_Oracle-LATAM/refs/heads/main/datos_tratados.csv"

**Fuente de Datos:**
- Los datos ya tratados provienen del siguiente URL: https://raw.githubusercontent.com/JosefRivers-Dev/Data-Science_Telecom_2_Oracle-LATAM/refs/heads/main/datos_tratados.csv

**Resumen del Proceso de Transformación de Datos:**
- **Normalización de Columnas:** Se renombraron las columnas para una mejor comprensión y se extrajeron los nombres finales después del último punto.
- **Cambio de Nombre a Español:** Se mapearon los nombres de las columnas a español para facilitar el análisis local.
- **Estandarización de Tipos de Datos:**
  - La columna 'Consumo_Total' se convirtió a tipo numérico, manejando posibles errores.
  - La columna 'Edad_Avanzada' se convirtió a tipo 'object'.
- **Limpieza y Estandarización de Valores:**
  - Se reemplazaron valores vacíos en 'Cancelacion_Servicio' por 'No'.
  - Se mapearon los valores numéricos (0, 1) en 'Edad_Avanzada' a ('No', 'Yes').
  - Se estandarizaron los valores 'No phone service' y 'No internet service' a 'No' en las columnas de servicios.